<a href="https://colab.research.google.com/github/jahnaviakurathi/machinelearning/blob/main/smartsurveillancesystemdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Import necessary libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Step 1: Frame Extraction
def extract_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames to {output_folder}.")

# Step 2: CNN Feature Extraction
def extract_cnn_features(frame_folder):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features = []

    for frame_file in sorted(os.listdir(frame_folder)):
        frame_path = os.path.join(frame_folder, frame_file)
        img = cv2.imread(frame_path)
        img = cv2.resize(img, (224, 224))
        img = preprocess_input(np.expand_dims(img, axis=0))
        feature = model.predict(img)
        features.append(feature.flatten())

    features = np.array(features)
    print(f"Extracted CNN features of shape: {features.shape}.")
    return features

# Step 3: Reshape Frame Features for RNN Input
def reshape_features_for_rnn(features, sequence_length):
    num_sequences = len(features) - sequence_length + 1
    rnn_input = np.array([features[i:i + sequence_length] for i in range(num_sequences)])
    return rnn_input

# Step 4: RNN Model Definition
def create_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        LSTM(32),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 5: Violence Detection Pipeline
def detect_violence(rnn_model, rnn_input, threshold=0.5):
    predictions = rnn_model.predict(rnn_input)
    flagged_frames = []

    for i, prob in enumerate(predictions):
        # Extract scalar from array (if prob is a single-element array)
        prob = prob.item() if isinstance(prob, np.ndarray) else prob

        if prob > threshold:
            flagged_frames.append(i)
            print(f"Violence detected in sequence {i} with probability {prob:.2f}.")

    return flagged_frames


# Main Function
if __name__ == "__main__":
    video_path = "/content/10.mp4"  # video file
    frame_folder = "frames"
    sequence_length = 10  # Number of frames per sequence

    # Step 1: Frame Extraction
    extract_frames(video_path, frame_folder)

    # Step 2: CNN Feature Extraction
    features = extract_cnn_features(frame_folder)

    # Step 3: Reshape Features for RNN Input
    rnn_input = reshape_features_for_rnn(features, sequence_length)
    print(f"RNN input shape: {rnn_input.shape}.")

    # Step 4: Train-Test Split (Mock labels for demonstration purposes)
    labels = np.random.randint(0, 2, size=(len(rnn_input),))  # Replace with real labels
    X_train, X_test, y_train, y_test = train_test_split(rnn_input, labels, test_size=0.2, random_state=42)

    # Define RNN model
    rnn_model = create_rnn_model(rnn_input.shape[1:])
    print(rnn_model.summary())

    # Train RNN model
    rnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=8)

    # Step 5: Violence Detection
    flagged_frames = detect_violence(rnn_model, X_test, threshold=0.5)
    print(f"Flagged sequences: {flagged_frames}")





Extracted 186 frames to frames.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/s

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 10, 64)              │         540,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 553,377 (2.11 MB)

 Trainable params: 553,377 (2.11 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.5250 - loss: 0.7540 - val_accuracy: 0.5556 - val_loss: 0.6874
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5021 - loss: 0.6909 - val_accuracy: 0.5556 - val_loss: 0.6905
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.4840 - loss: 0.6976 - val_accuracy: 0.5556 - val_loss: 0.6889
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5295 - loss: 0.6939 - val_accuracy: 0.5278 - val_loss: 0.6956
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6060 - loss: 0.6837 - val_accuracy: 0.4722 - val_loss: 0.6938
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5979 - loss: 0.6846 - val_accuracy: 0.5556 - val_loss: 0.6957
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.5191 - loss: 0.6840 - val_accuracy: 0.4444 - val_loss: 0.7013
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.6060 - loss: 0.6805 - val_accuracy: 0.527